In [1]:
from dotenv import load_dotenv
from IPython.lib.display import FileLink
import time
import httpx

load_dotenv()

True

In [2]:
# cd demos/fastapi_server 
# uv run uvicorn main:app --reload

In [3]:
from enum import Enum

BASE_URL = "http://localhost:8000"

class agent_type_enum(Enum):
    agent_no_tools = "agent_no_tools"
    agent_client_tools = "agent_client_tools"
    agent_all_raw = "agent_all_raw"
    agent_all_xml = "agent_all_xml"


def call_agent_api(
    prompt: str | list[dict] | dict,
    agent_uuid: str | None = None,
    agent_type: str | Enum = "agent_all_raw",
    log_file: str | None = None,
) -> str:
    """Call agent API and log streamed response to file.

    Returns:
        Path to the log file
    """
    if isinstance(agent_type, Enum):
        agent_type = agent_type.value

    # Build filename
    timestamp = int(time.time() * 1000)
    if log_file is None:
        log_file = f"temp_logs/{agent_type}_{timestamp}.log"
    else:
        log_file = f"temp_logs/{log_file}_{agent_type}_{timestamp}.log"

    # Build request payload
    payload = {"user_prompt": prompt}
    if agent_uuid:
        payload["agent_uuid"] = agent_uuid
    else:
        payload["agent_type"] = agent_type

    # Stream response and write to file
    with open(log_file, "w") as f:
        with httpx.Client(timeout=None) as client:
            with client.stream("POST", f"{BASE_URL}/agent/run", json=payload) as response:
                for line in response.iter_lines():
                    f.write(line)
                    f.write("\n")
                    # if line.startswith("data: "):
                        # data = line[6:]  # Strip "data: " prefix
                        # if data != "[DONE]":
                            # Unescape newlines and write
                            # f.write(data.replace("\\n", "\n"))
                            # f.write("\n")

    return log_file

In [ ]:
path = call_agent_api("Hello", log_file="hello", agent_type=agent_type_enum.agent_all_xml)
FileLink(path)

# Raw formatted Output

## No tools

In [ ]:
path = call_agent_api(
    "Hi, what is AI?", 
    log_file="no_tools_what_is_ai", 
    agent_type=agent_type_enum.agent_no_tools
)
FileLink(path)

## Client Tools

In [4]:
path = call_agent_api(
    "Hi, what is 5*(3+2/4)?", 
    log_file="client_tools_math_expression", 
    agent_type=agent_type_enum.agent_client_tools
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/client_tools_math_expression_agent_client_tools_1767179445495.log

## Server Tools

In [ ]:
call_agent_api(
    "Hi, what is 5*(3+2/4)?", 
    log_file="server_tools_raw_math_expression", 
    agent_type=agent_type_enum.agent_all_raw
)

In [ ]:
test_image = "../../currency_receipt_usd_jpy.png"
# Convert image to base64
import base64

with open(test_image, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_image,
            },
        },
        {
            "type": "text",
            "text": "I paid the receipt in USD. What is the amount in INR?"
        },
    ]
}
call_agent_api(
    prompt, 
    log_file="image_receipt_usd_to_inr", 
    agent_type=agent_type_enum.agent_all_raw
)

In [13]:
prompt = "Fetch me the latest happenings in the Y Combinator ecosystem"
path = call_agent_api(
    prompt,
    log_file="ycombinator_latest_happenings",
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/ycombinator_latest_happenings_agent_all_raw_1767180152784.log

## PDF citations

In [5]:
prompt = [{
    "type": "document",
    "source": {
        "type": "url",
        "url": "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"
    },
    "citations": {"enabled": True}
},
{
    "type": "text",
    "text": "What are the key findings in this document?"
}]

path = call_agent_api(
    prompt, 
    log_file="pdf_citations_model_card_key_findings", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/pdf_citations_model_card_key_findings_agent_all_raw_1767179560377.log

## Content citations

In [6]:
prompt = {
    "role": "user",
    "content": [
      {
        "type": "document",
        "source": {
          "type": "text",
          "media_type": "text/plain",
          "data": "The grass is green. The sky is blue."
        },
        "title": "My Document",
        "context": "This is a trustworthy document.",
        "citations": {"enabled": True}
        },
        {
        "type": "text",
        "text": "What color is the grass and sky?"
        }
    ]
    }
path = call_agent_api(
    prompt,
    log_file="content_citations_grass_sky",
    agent_type=agent_type_enum.agent_all_raw,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/content_citations_grass_sky_agent_all_raw_1767179579719.log

## Complex markdown Rendering Cases

In [ ]:
prompt = "Produce a markdown report with a table + code block + nested bullets about 'How to reduce latency in an API.'"
call_agent_api(
    prompt, 
    log_file="markdown_report_latency_in_api", 
    agent_type=agent_type_enum.agent_all_raw
)

## Image/File Generation

In [12]:
prompt = "Generate a JSON file named `users.json` with 20 synthetic users: id, name, email, country."
path = call_agent_api(
    prompt, 
    log_file="file_generation_users_json", 
    agent_type=agent_type_enum.agent_all_raw
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/file_generation_users_json_agent_all_raw_1767181854213.log

## Content Embedded Tags

In [ ]:
prompt = """
<chart_config>
**What Is It?**
A **Chart.js JSON config** is a text format that describes how your chart should look and what data it displays. Think of it like a recipe written in a specific format that Chart.js can read. The chart js json config can be used to show charts in the response.

**Basic Structure**:
```json
{
  "type": "bar",           // Chart type: 'bar', 'line', 'pie', 'doughnut', etc.
  "data": {                // Your chart data
    "labels": [...],       // X-axis labels (array of strings)
    "datasets": [...]      // Your data series (array of objects)
  },
  "options": {             // How the chart looks and behaves
    ...
  }
}
```

To show charts, embed Chart.js JSON config between `<chart>` tags.

**Important:** Place the JSON directly inside the tags without escaping or quotes:

```
<chart>
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2"],
    "datasets": [{"label": "My Data", "data": [10, 20]}]
  }
}
</chart>
```

**Complete Template**:
```json
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2", "Label 3"],
    "datasets": [{
      "label": "My Dataset",
      "data": [10, 20, 30],
      "backgroundColor": "rgba(75, 192, 192, 0.2)",
      "borderColor": "rgba(75, 192, 192, 1)",
      "borderWidth": 1
    }]
  },
  "options": {
    "responsive": true,
    "plugins": {
      "title": {
        "display": true,
        "text": "My Chart Title"
      }
    },
    "scales": {
      "y": {
        "beginAtZero": true
      }
    }
  }
}
```

**Rules**:
✅ **CAN include**: Numbers, Strings, Booleans, Arrays, Objects, Null

❌ **CANNOT include**:
- Functions or code or callbacks (config is serialized)
- Variables (like `myVariable`)
- Comments (remove `//` when saving as JSON)
- Single quotes (use double quotes only)
</chart_config>

<user_prompt>
Present world population demographic data in a chart.
</user_prompt>
"""
call_agent_api(
    prompt,
    log_file="content_embedded_tags_world_population_chart",
    agent_type=agent_type_enum.agent_all_raw,
)


# XML Formatted Output

## Server Tools

In [7]:
path = call_agent_api(
    "Hi, what is 5*(3+2/4)?",
    log_file="xml_server_tools_raw_math_expression",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_server_tools_raw_math_expression_agent_all_xml_1767179634197.log

In [8]:
test_image = "../../currency_receipt_usd_jpy.png"
# Convert image to base64
import base64

with open(test_image, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')

In [10]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_image,
            },
        },
        {
            "type": "text",
            "text": "I paid the receipt in USD. What is the amount in INR?"
        },
    ]
}
path = call_agent_api(
    prompt, 
    log_file="image_receipt_usd_to_inr", 
    agent_type=agent_type_enum.agent_all_xml
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/image_receipt_usd_to_inr_agent_all_xml_1767179740095.log

In [4]:
prompt = "Fetch me the latest happenings in the Y Combinator ecosystem"
path = call_agent_api(
    prompt,
    log_file="xml_ycombinator_latest_happenings",
    agent_type=agent_type_enum.agent_all_xml
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_ycombinator_latest_happenings_agent_all_xml_1767180928604.log

## PDF citations

In [6]:
prompt = [{
    "type": "document",
    "source": {
        "type": "url",
        "url": "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf",
    },
    "citations": {"enabled": True},
},
{
    "type": "text",
    "text": "What are the key findings in this document?",
}]

path = call_agent_api(
    prompt,
    log_file="xml_pdf_citations_model_card_key_findings",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_pdf_citations_model_card_key_findings_agent_all_xml_1767181061066.log

## Content citations

In [7]:
prompt = {
    "role": "user",
    "content": [
        {
            "type": "document",
            "source": {
                "type": "text",
                "media_type": "text/plain",
                "data": "The grass is green. The sky is blue.",
            },
            "title": "My Document",
            "context": "This is a trustworthy document.",
            "citations": {"enabled": True},
        },
        {
            "type": "text",
            "text": "What color is the grass and sky?",
        },
    ],
}

path = call_agent_api(
    prompt,
    log_file="xml_content_citations_grass_sky",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)


/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_content_citations_grass_sky_agent_all_xml_1767181085683.log

## Complex markdown Rendering Cases

In [8]:
prompt = "Produce a markdown report with a table + code block + nested bullets about 'How to reduce latency in an API.'"
path = call_agent_api(
    prompt,
    log_file="xml_markdown_report_latency_in_api",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_markdown_report_latency_in_api_agent_all_xml_1767181089814.log

## Image/File Generation

In [14]:
prompt = "Generate a JSON file named `users.json` with 20 synthetic users: id, name, email, country."
path = call_agent_api(
    prompt,
    log_file="xml_file_generation_users_json",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_file_generation_users_json_agent_all_xml_1767182210125.log

## Content Embedded Tags

In [10]:
prompt = """
<chart_config>
**What Is It?**
A **Chart.js JSON config** is a text format that describes how your chart should look and what data it displays. Think of it like a recipe written in a specific format that Chart.js can read. The chart js json config can be used to show charts in the response.

**Basic Structure**:
```json
{
  "type": "bar",           // Chart type: 'bar', 'line', 'pie', 'doughnut', etc.
  "data": {                // Your chart data
    "labels": [...],       // X-axis labels (array of strings)
    "datasets": [...]      // Your data series (array of objects)
  },
  "options": {             // How the chart looks and behaves
    ...
  }
}
```

To show charts, embed Chart.js JSON config between `<chart>` tags.

**Important:** Place the JSON directly inside the tags without escaping or quotes:

```
<chart>
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2"],
    "datasets": [{"label": "My Data", "data": [10, 20]}]
  }
}
</chart>
```

**Complete Template**:
```json
{
  "type": "bar",
  "data": {
    "labels": ["Label 1", "Label 2", "Label 3"],
    "datasets": [{
      "label": "My Dataset",
      "data": [10, 20, 30],
      "backgroundColor": "rgba(75, 192, 192, 0.2)",
      "borderColor": "rgba(75, 192, 192, 1)",
      "borderWidth": 1
    }]
  },
  "options": {
    "responsive": true,
    "plugins": {
      "title": {
        "display": true,
        "text": "My Chart Title"
      }
    },
    "scales": {
      "y": {
        "beginAtZero": true
      }
    }
  }
}
```

**Rules**:
✅ **CAN include**: Numbers, Strings, Booleans, Arrays, Objects, Null

❌ **CANNOT include**:
- Functions or code or callbacks (config is serialized)
- Variables (like `myVariable`)
- Comments (remove `//` when saving as JSON)
- Single quotes (use double quotes only)
</chart_config>

<user_prompt>
Present world population demographic data in a chart.
</user_prompt>
"""
path = call_agent_api(
    prompt,
    log_file="xml_content_embedded_tags_world_population_chart",
    agent_type=agent_type_enum.agent_all_xml,
)
FileLink(path)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/demos/fastapi_server/temp_logs/xml_content_embedded_tags_world_population_chart_agent_all_xml_1767181142849.log